In [1]:
#| label: libraries
#| include: false
import pandas as pd
import altair as alt
import numpy as np
import json

from IPython.display import Markdown
from IPython.display import display
from tabulate import tabulate

## Elevator pitch

_We got the information on delays at 7 airports over 10 years. After cleaning the data and wrangling it, I'll find what airport had the worst delays and which month is better to fly avoiding delays_


In [2]:
#| label: project data
#| code-summary: Read and format project data
# Include and execute your code here
dat = pd.read_json("https://raw.githubusercontent.com/byuidatascience/data4missing/master/data-raw/flights_missing/flights_missing.json")
#dat.dtypes



## Which airport has the worst delays?

__ Your answer should include a summary table that lists (for each airport) the total number of flights, total number of delayed flights, proportion of delayed flights, and average delay time in hours.__

_The Chicago O'Hare International Airport (ORD) is the airport with the worst delays. We determined this by counting the total hours of delays. The ORD had over 7,000 hours of delays, which is about 291 days of delay_


In [3]:
#pd.nunique('airport_code')

In [4]:
df = pd.DataFrame(dat)
#df.fillna()
#df.replace('', np.NaN)
#df.interpolate()
#CreateNaN's
df2=df.mask(df == '')


In [5]:
df2.replace(-999, np.nan, inplace=True)

#df2.replace('n/a', np.nan, inplace=True)

df2.replace('Febuary', 'February', inplace=True)

df2.replace('', np.nan, inplace=True)

df2.replace('1500+', 1500, inplace=True)

df2.query("month != 'n/a'")

# for counting official missing values df2.isna().sum()
# df2.dropna() for dropping na's
#df2


,airport_code,airport_name,month,year,num_of_flights_total,num_of_delays_carrier,num_of_delays_late_aircraft,num_of_delays_nas,num_of_delays_security,num_of_delays_weather,num_of_delays_total,minutes_delayed_carrier,minutes_delayed_late_aircraft,minutes_delayed_nas,minutes_delayed_security,minutes_delayed_weather,minutes_delayed_total
0,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",January,2005.0,35048,1500,NaN,4598,10,448,8355,116423.0,104415,207467.0,297,36931,465533
1,DEN,"Denver, CO: Denver International",January,2005.0,12687,1041,928.0,935,11,233,3153,53537.0,70301,36817.0,363,21779,182797
2,IAD,NaN,January,2005.0,12381,414,1058.0,895,4,61,2430,NaN,70919,35660.0,208,4497,134881
3,ORD,"Chicago, IL: Chicago O'Hare International",January,2005.0,28194,1197,2255.0,5415,5,306,9178,88691.0,160811,364382.0,151,24859,638894
4,SAN,"San Diego, CA: San Diego International",January,2005.0,7283,572,680.0,638,7,56,1952,27436.0,38445,21127.0,218,4326,91552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,DEN,"Denver, CO: Denver International",December,2015.0,18001,1001,1463.0,981,14,152,3610,64757.0,111638,60584.0,405,20271,257655
919,IAD,"Washington, DC: Washington Dulles International",December,2015.0,2799,182,183.0,61,0,17,443,NaN,15438,2826.0,0,1825,31164
920,ORD,"Chicago, IL: Chicago O'Hare International",December,2015.0,25568,923,1755.0,1364,11,180,4233,80962.0,132055,72045.0,435,22459,307956
922,SFO,"San Francisco, CA: San Francisco International",December,2015.0,13833,757,1180.0,2372,9,147,4465,55283.0,96703,193525.0,285,13788,359584


In [24]:
#dfna = df.isna().sum()
#dfna

airport_code                      0
airport_name                     56
month                            27
year                             23
num_of_flights_total              0
num_of_delays_carrier             0
num_of_delays_late_aircraft       0
num_of_delays_nas                 0
num_of_delays_security            0
num_of_delays_weather             0
num_of_delays_total               0
minutes_delayed_carrier          52
minutes_delayed_late_aircraft     0
minutes_delayed_nas              48
minutes_delayed_security          0
minutes_delayed_weather           0
minutes_delayed_total             0
dtype: int64

In [7]:

#Group and operate data
dft=df2.groupby('airport_code').agg(total_num_flights = ('num_of_flights_total', sum),
    total_delay_flights = ('num_of_delays_total', sum),
    average_delay_time = ('minutes_delayed_total', "mean")
).reset_index()


dft["proportion_delayed_flights"] = round(dft["total_delay_flights"] / dft["total_num_flights"],2)

dft["average_hr_delayed_total"] = round(dft["average_delay_time"] / 60,0)



#dft.dtypes

In [8]:
#dft.dtypes
dft['average_delay_time'] = dft['average_delay_time'].astype(int)
dft['average_hr_delayed_total'] = dft['average_hr_delayed_total'].astype(int)   

#better numeric format
dft['total_num_flights'] = dft['total_num_flights'].map('{:,d}'.format)
dft['total_delay_flights'] = dft['total_delay_flights'].map('{:,d}'.format)
dft['average_delay_time'] = dft['average_delay_time'].map('{:,d}'.format)
dft['average_hr_delayed_total'] = dft['average_hr_delayed_total'].map('{:,d}'.format)

# GRAND QUESTION 1
dft

,airport_code,total_num_flights,total_delay_flights,average_delay_time,proportion_delayed_flights,average_hr_delayed_total
0,ATL,"4,430,047","902,443","408,969",0.20,"6,816"
1,DEN,"2,513,974","468,519","190,707",0.19,"3,178"
2,IAD,"851,571","168,467","77,905",0.20,"1,298"
3,ORD,"3,597,588","830,825","426,940",0.23,"7,116"
4,SAN,"917,862","175,132","62,698",0.19,"1,045"
5,SFO,"1,630,945","425,604","201,140",0.26,"3,352"
6,SLC,"1,403,384","205,160","76,692",0.15,"1,278"


_include figures in chunks and discuss your findings in the figure._


In [9]:
#| label: GQ1 chart
#| code-summary: plot example
#| fig-align: center

chart_delay = (alt.Chart(dft,
    title = "Average delayed time by airport in hours")
    .encode(
    alt.X('airport_code'), 
    alt.Y('average_hr_delayed_total'),
    alt.Color('airport_code')
    )
    .mark_bar()
    )

display(chart_delay)

c:\Program Files\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

## GRAND QUESTION 2

__What is the best month to fly if you want to avoid delays of any length?__

_The best month to fly avoiding any kind of delay is September followed by November. To know that, we calculated the sum of the total of delays, and having the total number of flights, we divided it by the total of delays to get a proportion of totalflights/totaldelays. _


In [10]:
#| label: GQ2
#| code-summary: Read and format data
# Include and execute your code here

month_ord = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

dftm = df2.groupby(['airport_code','month']).agg(total_num_flights = ('num_of_flights_total', sum),
    total_delay_flights = ('num_of_delays_total', sum),
    average_delay_time = ('minutes_delayed_total', "mean")
).reset_index()


dftm["proportion_delayed_flights"] = round(dftm["total_delay_flights"] / dftm["total_num_flights"],2)

dftm["average_hr_delayed_total"] = round(dftm["average_delay_time"] / 60,2)

dftm

,airport_code,month,total_num_flights,total_delay_flights,average_delay_time,proportion_delayed_flights,average_hr_delayed_total
0,ATL,April,367596,63753,342467.545455,0.17,5707.79
1,ATL,August,389628,87883,494167.727273,0.23,8236.13
2,ATL,December,324731,69510,406413.800000,0.21,6773.56
3,ATL,February,301102,61603,359610.600000,0.20,5993.51
4,ATL,January,357513,70418,380031.727273,0.20,6333.86
...,...,...,...,...,...,...,...
86,SLC,May,105804,10984,53279.900000,0.10,888.00
87,SLC,November,108846,13148,56058.454545,0.12,934.31
88,SLC,October,115171,13320,54787.545455,0.12,913.13
89,SLC,September,101341,10390,49044.000000,0.10,817.40


In [11]:

#dftm1 = dftm.sort_values(month_ord)

#dft_month["month"].dropna()
#asd =asdf.dropna(subset = ['month'])

dftm.index = pd.CategoricalIndex(dftm.index, categories=month_ord, ordered=True)
dftm = dftm.sort_index()


dftm

,airport_code,month,total_num_flights,total_delay_flights,average_delay_time,proportion_delayed_flights,average_hr_delayed_total
NaN,ATL,April,367596,63753,342467.545455,0.17,5707.79
NaN,ATL,August,389628,87883,494167.727273,0.23,8236.13
NaN,ATL,December,324731,69510,406413.800000,0.21,6773.56
NaN,ATL,February,301102,61603,359610.600000,0.20,5993.51
NaN,ATL,January,357513,70418,380031.727273,0.20,6333.86
...,...,...,...,...,...,...,...
NaN,SLC,May,105804,10984,53279.900000,0.10,888.00
NaN,SLC,November,108846,13148,56058.454545,0.12,934.31
NaN,SLC,October,115171,13320,54787.545455,0.12,913.13
NaN,SLC,September,101341,10390,49044.000000,0.10,817.40


_include figures in chunks and discuss your findings in the figure._


In [12]:
month_chart = (alt.Chart(dftm,
    title = "Total delays by month")
    .encode(
    alt.X('month', sort=month_ord), 
    alt.Y('proportion_delayed_flights'),
    alt.Color('airport_code')
    )
    .mark_bar()
    )
    
display(month_chart)

c:\Program Files\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

## GRAND QUESTION 3

_Your job is to create a new column that calculates the total number of flights delayed by weather (both severe and mild)._
_Using the new weather variable calculated above, create a barplot showing the proportion of all flights that are delayed by weather at each airport. Discuss what you learn from this graph._

_type your results and analysis here_


In [13]:
df.replace(-999, np.nan, inplace=True)

df.replace('n/a', np.nan, inplace=True)

df.replace('Febuary', 'February', inplace=True)

df.replace('', np.nan, inplace=True)

df.replace('1500+', 1500, inplace=True)

df.query("month != 'na'")

air_mean = df['num_of_delays_late_aircraft'].mean()
df.num_of_delays_late_aircraft.replace(np.nan, air_mean, inplace = True)

In [14]:
#| label: GQ3
#| code-summary: Read and format data
# Include and execute your code here

df_weather = df.assign(
mild_weather_delays = df.num_of_delays_late_aircraft * 0.3,
nas_weather_delays = np.where(
    df.month.isin(['April', 'May', 'June', 'July' 'August']),
    df.num_of_delays_nas * 0.4,
    df.num_of_delays_nas * 0.65
)
)

df_weather

,airport_code,airport_name,month,year,num_of_flights_total,num_of_delays_carrier,num_of_delays_late_aircraft,num_of_delays_nas,num_of_delays_security,num_of_delays_weather,num_of_delays_total,minutes_delayed_carrier,minutes_delayed_late_aircraft,minutes_delayed_nas,minutes_delayed_security,minutes_delayed_weather,minutes_delayed_total,mild_weather_delays,nas_weather_delays
0,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",January,2005.0,35048,1500,1109.104072,4598,10,448,8355,116423.0,104415,207467.0,297,36931,465533,332.731222,2988.70
1,DEN,"Denver, CO: Denver International",January,2005.0,12687,1041,928.000000,935,11,233,3153,53537.0,70301,36817.0,363,21779,182797,278.400000,607.75
2,IAD,NaN,January,2005.0,12381,414,1058.000000,895,4,61,2430,NaN,70919,35660.0,208,4497,134881,317.400000,581.75
3,ORD,"Chicago, IL: Chicago O'Hare International",January,2005.0,28194,1197,2255.000000,5415,5,306,9178,88691.0,160811,364382.0,151,24859,638894,676.500000,3519.75
4,SAN,"San Diego, CA: San Diego International",January,2005.0,7283,572,680.000000,638,7,56,1952,27436.0,38445,21127.0,218,4326,91552,204.000000,414.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,IAD,"Washington, DC: Washington Dulles International",December,2015.0,2799,182,183.000000,61,0,17,443,NaN,15438,2826.0,0,1825,31164,54.900000,39.65
920,ORD,"Chicago, IL: Chicago O'Hare International",December,2015.0,25568,923,1755.000000,1364,11,180,4233,80962.0,132055,72045.0,435,22459,307956,526.500000,886.60
921,SAN,"San Diego, CA: San Diego International",NaN,2015.0,6231,480,606.000000,256,5,37,1383,25402.0,35796,9038.0,161,2742,73139,181.800000,166.40
922,SFO,"San Francisco, CA: San Francisco International",December,2015.0,13833,757,1180.000000,2372,9,147,4465,55283.0,96703,193525.0,285,13788,359584,354.000000,1541.80


In [15]:
weather_total = df_weather.assign(total_weather_delays = lambda x: x.num_of_delays_weather + x.mild_weather_delays + x.nas_weather_delays,
    weather_proportion = lambda x: x.total_weather_delays / x.num_of_delays_total
).filter(['airport_code','month', 'num_of_delays_weather','mild_weather_delays','nas_weather_delays', 'total_weather_delays', 'weather_proportion']
)

weather_total

,airport_code,month,num_of_delays_weather,mild_weather_delays,nas_weather_delays,total_weather_delays,weather_proportion
0,ATL,January,448,332.731222,2988.70,3769.431222,0.451159
1,DEN,January,233,278.400000,607.75,1119.150000,0.354948
2,IAD,January,61,317.400000,581.75,960.150000,0.395123
3,ORD,January,306,676.500000,3519.75,4502.250000,0.490548
4,SAN,January,56,204.000000,414.70,674.700000,0.345645
...,...,...,...,...,...,...,...
919,IAD,December,17,54.900000,39.65,111.550000,0.251806
920,ORD,December,180,526.500000,886.60,1593.100000,0.376352
921,SAN,NaN,37,181.800000,166.40,385.200000,0.278525
922,SFO,December,147,354.000000,1541.80,2042.800000,0.457514


_include figures in chunks and discuss your findings in the figure._


In [16]:

# Include and execute your code here
weather_proportion_chart = (alt.Chart(weather_total.
    query("month != 'null'"),
    title = "Proportion of Weather delays by airport")
    .encode(
    alt.X('airport_code', sort=month_ord), 
    alt.Y('weather_proportion'),
    alt.Color('airport_code')
    )
    .mark_bar()
    )
    
display(weather_proportion_chart)

c:\Program Files\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

From the graph above we get that San Francisco Airport (SFO) was the airport with more delays because of weather in proportion to its total flights. And being San Antonio Airport (SAN) the best with less delays. Even both airports are in the state of California, we'd think they are not that different, but googling around about the weather in these two cities, San Diego is warmer and San Francisco is known for its wind and fog during summer.

## APPENDIX A (Additional Python Code)

```python
#paste other your code from your python file (.py) here
```

_Fix all of the varied missing data types in the data to be consistent (all missing values should be displayed as “NaN”)_


Questions 4 and 5:
4.- Using the new weather variable calculated above, create a barplot showing the proportion of all flights that are delayed by weather at each airport. Discuss what you learn from this graph.

5.-  Fix all of the varied missing data types in the data to be consistent (all missing values should be displayed as “NaN”). In your report include one record example (one row) from your new data, in the raw JSON format. Your example should display the “NaN” for at least one missing value.

Question 4 was responded by creating the barplot in the section of question 3. 
Question 5 is responded at the beginning of the project to clean the data from missing values, errors, etc. The tables shared in Q1 and Q2 are proof of this.